# [Node.js] async 라이브러리 - Control Flow

### applyEach 함수

```jsx
applyEach(fns, …args(opt), callback(opt))
```

주어진 인자들을 함수 배열의 각 함수에 적용한 후, 모든 함수가 완료되면 콜백을 호출하는 함수

| Name | Type | Description |
| --- | --- | --- |
| fns | Array, Iterable , AsyncIterable , Object | 비동기 함수들의 배열이나 컬렉션입니다. 이 함수들은 모두 동일한 인자들을 받아 호출됩니다. |
| args | * <optional> | 함수들에 전달될 인자들입니다. 이 인자들은 각 함수에 별도로 전달됩니다. |
| callback | function <optional> | 모든 함수의 처리가 완료되었을 때 호출되는 콜백 함수입니다. 이 콜백은 각 함수의 결과를 배열로 받아 처리합니다. |

**전달하는 인수(args)가 없을 경우**

- **함수 반환 →** 첫 번째 인자(**`fns`**)만 제공되면, **`async/applyEach`**는 인자들을 적용할 준비가 된 함수를 반환합니다. 이 반환된 함수는 나중에 호출될 때 인자들을 받을 수 있습니다.

**전달하는 인수(args)가 있을 경우**

- **직접 호출 →** 함수 배열과 함께 인자들을 제공하면, **`async/applyEach`**는 이 인자들을 각 함수에 적용하고, 모든 함수가 완료되면 콜백을 호출합니다.

#### **주의할 점**

##### 1. **넘겨주는 인자와, 함수의 인자 개수가 일치해야 한다.**

In [1]:
const async=require('async');

In [2]:
function firstFunction(arg1, arg2, callback) {
    setTimeout(function () {
        console.log('First function:', arg1);
        callback(null, arg1);
    }, 1000);
}
function secondFunction(arg1, arg2, callback) {
    setTimeout(function () {
        console.log('Second function:', arg2);
        callback(null, arg2);
    }, 2000);
}
const func = async.applyEach([firstFunction, secondFunction], 1, 2, function (err, result) {
    if (err) {
        console.log(err)
    } else {
        console.log(result)
    }
});
func();

Promise { <pending> }

First function: 1
1
Second function: 2
2


이렇게 인자 (1,2) 2개를 넘겨주고 싶다면 받는 함수의 인자 개수도 2개가 되어야한다. 만일

In [5]:
function firstFunction1(arg, callback) {
    setTimeout(function () {
        console.log('First function:', arg1);
        callback(null, arg);
    }, 1000);
}
function secondFunction1(arg, callback) {
    setTimeout(function () {
        console.log('Second function:', arg2);
        callback(null, arg);
    }, 2000);
}

const func1 = async.applyEach([firstFunction1, secondFunction1], 1, 2, function (err, result) {
    if (err) {
        console.log(err)
    } else {
        console.log(result)
    }
});

SyntaxError: Identifier 'func1' has already been declared

이렇게 함수 파라미터 개수가 일치하지 않으면 오류가 발생한다.

##### 2. **applyEach 는 함수를 반환하는 함수다.** 
    
`async.applyEach`는 다른 `async` 모듈 함수들과 달리, 즉시 비동기 작업을 시작하는 것이 아니라, 실행할 준비가 된 **함수를 반환하는 함수**다. `async.applyEach`를 호출하면, 그 결과로 함수가 반환되기 때문에 반환된 함수는 나중에 호출되어야 비동기 작업을 시작한다. 
    
함수호출 방법엔 아래 2가지 방법이 있다.

2-1. **변수에 함수 할당 후 호출 (3-1의 경우)**

반환된 함수를 변수에 할당하고, 이 변수를 사용하여 함수를 나중에 호출할 수 있다

In [6]:
const appliedFunctions = async.applyEach([firstFunction, secondFunction], 1, 2);

appliedFunctions(function(err, results) {
    // 콜백 로직
});

First function: 1


TypeError: callback is not a function
    at Timeout._onTimeout (evalmachine.<anonymous>:4:9)
    at listOnTimeout (node:internal/timers:573:17)
    at process.processTimers (node:internal/timers:514:7)

Second function: 2


여기서 `appliedFunctions`는 `async.applyEach`에 의해 반환된 함수다. 이 함수는 비동기 작업을 시작할 준비가 되었으며, 실제로 작업을 시작하려면 `appliedFunctions()`를 호출해야한다

2-2. . **바로 호출 (`()`를 사용) (3-2의 경우)**

반환된 함수를 바로 호출할 수도 있다. 이는 `async.applyEach`의 호출과 반환된 함수의 호출을 한 줄로 결합한다.

In [ ]:
async.applyEach([firstFunction, secondFunction], 1, 2, function(err, results) {
    // 콜백 로직
})();

이 코드는 `async.applyEach`가 반환한 함수를 바로 호출합니다. `()`는 바로 함수를 호출하는 JavaScript의 표준 문법이다. <br/>
결론적으로, `async.applyEach`는 즉시 실행되는 것이 아니라, 나중에 호출할 수 있는 함수를 반환한다. 이 함수는 비동기 작업을 시작하며, 작업이 완료되면 제공된 콜백을 실행한다.

##### 3. **콜백함수를 파라미터에 넣냐vs안넣냐에 따라 실행 순서가 달라진다**
    
콜백함수가 optional이다. 콜백함수가 있냐/없냐, 파라미터에 포함이 되냐/안되냐에 따라서 실행순서가 달라지고, blocking 되는 것이 달라진다.
    
3-1. 콜백함수가 따로 선언이 되는 경우 (파라미터에 포함 X)

In [7]:
// 비동기 함수들 정의
function firstFunction(arg, callback) {
    setTimeout(function () {
        console.log('First function:', arg[0]);
        callback(null, arg[0]);
    }, 1000);
}

function secondFunction(arg, callback) {
    setTimeout(function () {
        console.log('Second function:', arg[1]);
        callback(null, arg[1]);
    }, 1000);
}

// applyEach를 사용하여 두 함수에 적용할 새로운 함수 생성
const applyFunctions = async.applyEach([firstFunction, secondFunction], [1,2]);

// 생성된 함수 호출
applyFunctions(function (err, results) {
    if (err) {
        console.log('Error:', err);
    } else {
        console.log('Results:', results);
    }
});

First function: 1
Second function: 2
Results: [ 1, 2 ]


이렇게 콜백 함수를 따로 작성하면, 배열안에 있는 함수가 전부 실행된 후에 콜백함수가 실행된다. 위 예시의 경우, `firstFunction`과 `secondFunction`이 모두 실행이 된 다음의 각각의 결과값을 배열로 받아 `results`로 콘솔에 출력한다.

3-2. 콜백함수가 파라미터에 포함되는 경우

In [8]:
function firstFunction(arg1, arg2, callback) {
    setTimeout(function () {
        console.log('First function:', arg1);
        callback(null, arg1);
    }, 1000);
}
function secondFunction(arg1, arg2, callback) {
    setTimeout(function () {
        console.log('Second function:', arg2);
        callback(null, arg2);
    }, 1000);
}

// applyEach를 사용하여 두 함수를 호출
async.applyEach([firstFunction, secondFunction], 1, 2, function (err, result){
    if (err) {
        console.log(err)
    } else {
        console.log(result)
    }
})();

Promise { <pending> }

First function: 1
1
Second function: 2
2


콜백 함수가 파라미터에 포함되면, 배열 안 함수가 모두 호출 된 다음 콜백 함수가 실행되는 것이 아니라, 각각의 함수가 실행한 직후 콜백 함수가 바로 실행된다. 위 예시의 경우 `firstFunction`이 완료되면 콜백함수가 실행되고, `secondFunction`이 완료된 후 또 콜백 함수가 실행된다. 그럼 각각의 `result`값이 따로 콘솔에 출력되는 것이다. <br/>

여기서 `blocking`의 차이가 발생하는데, 3-1(콜백함수가 파라미터에 포함되지 않는) 경우 전체 함수가 실행된 후 콜백함수가 실행되기 때문에 전체 함수가 실행되기 전까지는 콜백함수에 `blocking`이 되고 `secondFunction`에는 `non-blocking`이다. 하지만 3-2(콜백함수가 파라미터에 포함되는) 경우 각각 함수가 실행된 후 콜백함수가 실행되기 때문에 `firstFunction`이 실행되는 도중 `secondFunction`에 `blocking`이 되고, 콜백함수에는 `non-blocking`이 되는 차이점이 있다.